In [1]:
import pandas as pd

- Let's import our clean dataset

In [2]:
sent2 = pd.read_pickle('data/sent2.pkl')

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.preprocessing import text
from tensorflow.keras import utils

train_size = int(len(sent2) * .7)
train_posts = sent2['posts'][:train_size]
train_types = sent2['type'][:train_size]

test_posts = sent2['posts'][train_size:]
test_types = sent2['type'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_types)

encoder = LabelEncoder()
encoder.fit(train_types)
y_train = encoder.transform(train_types)
y_test = encoder.transform(test_types)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [4]:
batch_size = 500
epochs = 15

# Build the model
model = Sequential()
model.add(Dense(500, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

W0702 11:48:59.296010 4350109120 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5464 samples, validate on 608 samples
Epoch 1/15
5464/5464 [==============================] - 1s 155us/sample - loss: 2.5009 - accuracy: 0.1944 - val_loss: 2.1926 - val_accuracy: 0.2944
Epoch 2/15
5464/5464 [==============================] - 0s 72us/sample - loss: 2.1116 - accuracy: 0.3042 - val_loss: 2.0739 - val_accuracy: 0.3487
Epoch 3/15
5464/5464 [==============================] - 0s 72us/sample - loss: 1.9175 - accuracy: 0.3889 - val_loss: 1.9692 - val_accuracy: 0.3750
Epoch 4/15
5464/5464 [==============================] - 0s 72us/sample - loss: 1.7468 - accuracy: 0.4484 - val_loss: 1.8975 - val_accuracy: 0.3914
Epoch 5/15
5464/5464 [==============================] - 0s 72us/sample - loss: 1.5903 - accuracy: 0.4989 - val_loss: 1.8529 - val_accuracy: 0.4112
Epoch 6/15
5464/5464 [==============================] - 0s 72us/sample - loss: 1.4481 - accuracy: 0.5404 - val_loss: 1.8091 - val_accuracy: 0.4178
Epoch 7/15
5464/5464 [==============================] - 0s 74us/sample

In [5]:
y_pred = model.predict_classes(x_test)

In [6]:
y_test2 =[]
for i in range(len(y_test)):
    y_test2.append(y_test[i].argmax())

In [8]:
from sklearn.metrics import accuracy_score,classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test2))
print(classification_report(y_test2, y_pred))
print(confusion_matrix(y_test2,y_pred))

accuracy 0.9865539761813292
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       1.00      1.00      1.00       214
           2       1.00      1.00      1.00        77
           3       1.00      1.00      1.00       186
           4       0.00      0.00      0.00        11
           5       0.00      0.00      0.00        15
           6       0.00      0.00      0.00         9
           7       1.00      1.00      1.00        21
           8       1.00      1.00      1.00       425
           9       0.98      1.00      0.99       538
          10       0.96      1.00      0.98       327
          11       0.97      1.00      0.99       408
          12       1.00      1.00      1.00        53
          13       1.00      1.00      1.00        88
          14       1.00      1.00      1.00        69
          15       1.00      1.00      1.00       108

   micro avg       0.99      0.99      0.99      260

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
mykey= {0:'ENFJ',1:'ENFP',2:'ENTJ',3:'ENTP',4:'ESFJ',5:'ESFP',6:'ESTJ',7:'ESTP',8:"INFJ",9: 'INFP',10:'INTJ',11:'INTP',12:'ISFJ',13:'ISFP',14:'ISTJ',15:'ISTP'}